In [59]:
import math
import shutil
import random
import argparse
from scipy import ndimage
import numpy as np
import cv2, imageio
from skimage.exposure import match_histograms
import matplotlib.pyplot as plt
from IPython.display import Image
import os, subprocess

In [11]:
def prewitt(img):
    img_gaussian = cv2.GaussianBlur(img, (3,3), 0)
    kernelx = np.array( [[1, 1, 1],[0, 0, 0],[-1, -1, -1]] )
    kernely = np.array( [[-1, 0, 1],[-1, 0, 1],[-1, 0, 1]] )
    img_prewittx = cv2.filter2D(img_gaussian, -1, kernelx)
    img_prewitty = cv2.filter2D(img_gaussian, -1, kernely)
    return img_prewittx // 15.36, img_prewitty // 15.36

def roberts(img):
    roberts_cross_v = np.array([[ 0, 0, 0 ],
                                 [ 0, 1, 0 ],
                                 [ 0, 0,-1 ]])
    roberts_cross_h = np.array([[ 0, 0, 0 ],
                                 [ 0, 0, 1 ],
                                 [ 0,-1, 0 ]])
    vertical = ndimage.convolve( img, roberts_cross_v )
    horizontal = ndimage.convolve( img, roberts_cross_h )
    return vertical // 50.0, horizontal // 50.0

# Different Edge Operator 
def get_gradient(img_o, ksize, gtype):
    if gtype == 'prewitt':
        X, Y = prewitt(img_o)
    elif gtype == 'sobel':
        X = cv2.Sobel(img_o,cv2.CV_32F,1,0,ksize=5)  / 50.0
        Y = cv2.Sobel(img_o,cv2.CV_32F,0,1,ksize=5)  / 50.0
    elif gtype == 'roberts':
        X, Y = roberts(img_o)
    else:
        print('Not suppported type!')
        exit()

    # Blur the Gradient to smooth the edge
    X = cv2.GaussianBlur(X, ksize, 0)
    Y = cv2.GaussianBlur(Y, ksize, 0)
    return X, Y

def draw_order(h, w, scale):
    order = []
    for i in range(0, h, scale):
        for j in range(0, w, scale):
            y = random.randint(-scale // 2, scale // 2) + i
            x = random.randint(-scale // 2, scale // 2) + j
            order.append((y % h, x % w))
    return order

In [60]:
def transfer_image(img_paths, edge_detect_method, brush_width, output_path='outputs/images', hist_spec=False):
    brush_width = int(brush_width)
    quant = 0 #float(args.palette) Color come from palette(limited colours)

    for idx, img_path in enumerate(img_paths):
    
        img_path_split = img_path.rsplit(".", -1) 
        imgs_path = [img_path]

        # Process the image
        result = []
        for path in imgs_path:
            print("\n" + path)
            img = cv2.imread(path)
            
            if hist_spec == True:
                reference = cv2.imread('spec.jpg')
                matched = match_histograms(img, reference, multichannel=True)
                img = matched
            
            # Get the gradient of image (Using sobel, scharr, prewitt, or roberts)
            print("Gradient: ");
            r = 2 * int(img.shape[0] / 50) + 1
            Gx, Gy = get_gradient(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (r, r), edge_detect_method)
            Gh = np.sqrt(np.sqrt(np.square(Gx) + np.square(Gy)))    # Length of the ellipse
            Ga = (np.arctan2(Gy, Gx) / np.pi) * 180 + 90            # Angle of the ellipse

            print("Drawing");
            canvas = cv2.medianBlur(img, 11)    # Make the image artlistic
            order = draw_order(img.shape[0], img.shape[1], scale=brush_width*2)

            # Draw the ellipse
            colors = np.array(img, dtype=np.float)
            for i, (y, x) in enumerate(order):
                length = int(round(brush_width + brush_width * Gh[y, x]))
                color = colors[y, x]

                cv2.ellipse(canvas, (x, y), (length, brush_width), Ga[y, x], 0, 360, color, -1, cv2.LINE_AA)

            result.append(canvas)

        # Output the result
        print("\nOutput the result")
        _output_path = "{}/{}".format(output_path, '{}_{}.jpg'.format(idx + 1, edge_detect_method))
        print(_output_path)
    #     if img_path[1] == 'gif':
    #         c, images = 0, []
    #         for canva in result:
    #             cv2.imwrite(img_path[0]+'/r-'+str(c)+'.jpg', canva); c += 1
    #         for i in range(c-1): images.append(imageio.imread(img_path[0]+'/r-'+str(i)+'.jpg'))
    #         from PIL import Image
    #         imageio.mimsave(output_path, images, duration=float(Image.open(args.path).info['duration']) / 1000)
    #         shutil.rmtree(img_path[0], ignore_errors=True)
    #     else:
        cv2.imwrite(_output_path, result[0])
        print("{} done".format(img_path_split[0]))
    
    print("done!")

In [46]:
input_path = 'images';
image_paths = []
for file_name in os.listdir(input_path):
    if file_name.endswith(".jpg"):
        image_paths.append("{}/{}".format(input_path, file_name))

transfer_image(image_paths, 'prewitt', 2, 'outputs/prewitt', equalize=False)


images/4.jpg
Gradient: 
Drawing

Output the result
outputs/prewitt/1_prewitt.jpg
images/4 done

images/5.jpg
Gradient: 
Drawing

Output the result
outputs/prewitt/2_prewitt.jpg
images/5 done

images/2.jpg
Gradient: 
Drawing

Output the result
outputs/prewitt/3_prewitt.jpg
images/2 done

images/3.jpg
Gradient: 
Drawing

Output the result
outputs/prewitt/4_prewitt.jpg
images/3 done

images/1.jpg
Gradient: 
Drawing

Output the result
outputs/prewitt/5_prewitt.jpg
images/1 done
done!


In [63]:
input_path = 'images';
image_paths = []
for file_name in os.listdir(input_path):
    if file_name.endswith(".jpg"):
        image_paths.append("{}/{}".format(input_path, file_name))

transfer_image(image_paths, 'sobel', 5, 'outputs/hist_spec', hist_spec=True)


images/4.jpg
Gradient: 
Drawing

Output the result
outputs/hist_spec/1_sobel.jpg
images/4 done

images/5.jpg
Gradient: 
Drawing

Output the result
outputs/hist_spec/2_sobel.jpg
images/5 done

images/2.jpg
Gradient: 
Drawing

Output the result
outputs/hist_spec/3_sobel.jpg
images/2 done

images/3.jpg
Gradient: 
Drawing

Output the result
outputs/hist_spec/4_sobel.jpg
images/3 done

images/1.jpg
Gradient: 
Drawing

Output the result
outputs/hist_spec/5_sobel.jpg
images/1 done
done!
